In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------
# 1) 데이터 불러오기
# -----------------------------

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 20s 0us/step


In [2]:
# y의 shape을 (N,)으로 변환
y_train = y_train.reshape(-1)
y_test  = y_test.reshape(-1)

print(f"[원본] x_train: {x_train.shape}, x_test: {x_test.shape}")

[원본] x_train: (50000, 32, 32, 3), x_test: (10000, 32, 32, 3)


In [3]:
# -----------------------------
# 2) 클래스 이름 정의
# -----------------------------

class_names_full = ['airplane','automobile','bird','cat','deer',
                    'dog','frog','horse','ship','truck']

In [4]:
# -----------------------------
# 3) 3개의 클래스 선택
# -----------------------------

selected_classes = ['cat','dog','horse']

selected_idx = [class_names_full.index(c) for c in selected_classes]

In [5]:
# -----------------------------
# 4) 선택한 클래스만 필터링
# -----------------------------

train_mask = np.isin(y_train, selected_idx)
test_mask  = np.isin(y_test,  selected_idx)

x_train = x_train[train_mask]
y_train = y_train[train_mask]
x_test  = x_test[test_mask]
y_test  = y_test[test_mask]

In [6]:
# 라벨을 0~2로 매핑
label_map = {orig:i for i, orig in enumerate(selected_idx)}
y_train = np.array([label_map[y] for y in y_train], dtype=np.int64)
y_test  = np.array([label_map[y] for y in y_test],  dtype=np.int64)

print(f"[선택] 클래스: {selected_classes}")
print(f"[선택] 학습샘플: {len(x_train)}, 테스트샘플: {len(x_test)}")

[선택] 클래스: ['cat', 'dog', 'horse']
[선택] 학습샘플: 15000, 테스트샘플: 3000


In [7]:
# -----------------------------
# 5) 정규화 (0~1) 및 텐서 변환
# -----------------------------

x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

In [8]:
# -----------------------------
# 6) CNN 모델 구성
# -----------------------------

model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),

    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),                    # 32 -> 16

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),                    # 16 -> 8

    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),                    # 8 -> 4

    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')           # 3개 클래스 (cat, dog, horse)
])

In [9]:
# -----------------------------
# 7) 모델 컴파일
# -----------------------------

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [10]:
# 모델 요약 정보 출력
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,907 (1.36 MB)

 Trainable params: 355,907 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# -----------------------------
# 8) 모델 학습
# -----------------------------

history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    verbose=2
)

Epoch 1/10
106/106 - 10s - 95ms/step - accuracy: 0.5224 - loss: 0.9460 - val_accuracy: 0.5787 - val_loss: 0.8573
Epoch 2/10
106/106 - 1s - 8ms/step - accuracy: 0.6373 - loss: 0.7801 - val_accuracy: 0.6560 - val_loss: 0.7456
Epoch 3/10
106/106 - 1s - 7ms/step - accuracy: 0.6808 - loss: 0.6994 - val_accuracy: 0.6927 - val_loss: 0.6803
Epoch 4/10
106/106 - 1s - 7ms/step - accuracy: 0.7199 - loss: 0.6345 - val_accuracy: 0.7133 - val_loss: 0.6553
Epoch 5/10
106/106 - 1s - 7ms/step - accuracy: 0.7428 - loss: 0.5922 - val_accuracy: 0.7313 - val_loss: 0.6049
Epoch 6/10
106/106 - 1s - 7ms/step - accuracy: 0.7610 - loss: 0.5558 - val_accuracy: 0.7533 - val_loss: 0.5955
Epoch 7/10
106/106 - 1s - 7ms/step - accuracy: 0.7735 - loss: 0.5262 - val_accuracy: 0.7440 - val_loss: 0.5871
Epoch 8/10
106/106 - 1s - 7ms/step - accuracy: 0.7889 - loss: 0.4963 - val_accuracy: 0.7507 - val_loss: 0.5929
Epoch 9/10
106/106 - 1s - 7ms/step - accuracy: 0.8044 - loss: 0.4647 - val_accuracy: 0.7780 - val_loss: 0.5418

In [12]:
# -----------------------------
# 9) 모델 평가
# -----------------------------

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f} | Test Loss: {test_loss:.4f}")

Test Accuracy: 0.7697 | Test Loss: 0.5374


In [13]:
# -----------------------------
# 10) 모델 예측
# -----------------------------

y_pred_prob = model.predict(x_test, verbose=0)
y_pred = np.argmax(y_pred_prob, axis=1)

In [14]:
# -----------------------------
# 11) 평가 지표: 레포트/혼동행렬
# -----------------------------

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=selected_classes, digits=4))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

         cat     0.7305    0.7290    0.7297      1000
         dog     0.6915    0.7420    0.7159      1000
       horse     0.9020    0.8380    0.8688      1000

    accuracy                         0.7697      3000
   macro avg     0.7747    0.7697    0.7715      3000
weighted avg     0.7747    0.7697    0.7715      3000

Confusion Matrix:
[[729 235  36]
 [203 742  55]
 [ 66  96 838]]


In [15]:
# 샘플 예측 확인
sample_idx = np.arange(10)
sample_prob = model.predict(x_test[sample_idx], verbose=0)
sample_pred = np.argmax(sample_prob, axis=1)

inv_label_map = {v:k for k,v in label_map.items()}  # 0/1/2 -> 원래 CIFAR index
pred_names = [selected_classes[i] for i in sample_pred]
true_names = [selected_classes[y_test[i]] for i in sample_idx]

print("샘플 예측:", pred_names)
print("실제 레이블:", true_names)

샘플 예측: ['cat', 'cat', 'cat', 'horse', 'dog', 'horse', 'horse', 'dog', 'dog', 'dog']
실제 레이블: ['cat', 'cat', 'dog', 'horse', 'dog', 'horse', 'horse', 'dog', 'dog', 'dog']
